# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# first examine data 
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='w'].call))
print("black-sounding names", sum(data[data.race=='b'].call))

235.0
black-sounding names 157.0


In [6]:
#determine total number of resumes
data.describe()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,...,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind
count,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,...,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000
mean,3.618480,3.661396,7.842916,0.052772,0.411499,0.097125,0.448049,215.637782,3.481520,0.559548,...,0.106776,0.437166,0.072690,0.082957,0.030390,0.085010,0.213963,0.267762,0.154825,0.165092
std,0.714997,1.219126,5.044612,0.223601,0.492156,0.296159,0.497345,148.127551,2.038036,0.496492,...,0.308866,0.496083,0.259649,0.275854,0.171677,0.278932,0.410141,0.442847,0.361773,0.371308
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,27.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,267.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,4.000000,9.000000,0.000000,1.000000,0.000000,1.000000,313.000000,6.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,4.000000,7.000000,44.000000,1.000000,1.000000,1.000000,1.000000,903.000000,6.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 1.)What test is appropriate for this problem? Does CLT apply?
### CLT states that the mean of all samples from the same population will be approximately equal to the mean of the population given a sufficiently large sample size. The cell above indicates 4870 samples(resumes) which is sufficiently large.  Now we need to look at the detailed racial stats to decide which test.

In [7]:
#determine number of white and black resumes.
w = data[data.race=='w']
b = data[data.race=='b']
print('Black Resumes: ', len(b))
print('White Resumes: ', len(w))


Black Resumes:  2435
White Resumes:  2435


### Since this is a perfect split in half, two sample hypothesis testing is what is called for.  Two sample hypothesis testing will allow us to test if there is a difference in means of the two different populations

## 2.) What are the null and alternate hypotheses?
### null hypothesis: Mean(Black Callbacks) - Mean(White Callbacks) = 0
### alt hypothesis:  Mean(Black Callbacks) - Mean(White Callbacks) != 0

## 3.)Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches. 1 means resume receives a callback and 0 means resume does not receive a callback.



In [43]:
#first determine means, std, as well as the difference in means to see if it equals zero and to use in CI calculation.
print('Total Callback mean:', data['call'].mean())
print('Total Callback std:', data['call'].std())
nullhyp_diff_of_means = 0
print('Black Callback Mean:  {0:.3f}'.format(b['call'].mean()))
print('White Callback Mean:  {0:.3f}'.format(w['call'].mean()))
black_std = b['call'].std()
white_std = w['call'].std()
print('Black Callback std:  {0:.3f}'.format(black_std))
print('White Callback std:  {0:.3f}'.format(white_std))
diff_of_means = (b['call'].mean()) - (w['call'].mean())
print('Diff of Means:  {0:.3f}'.format(b['call'].mean() - w['call'].mean()))

Total Callback mean: 0.08049281686544418
Total Callback std: 0.2720794379711151
Black Callback Mean:  0.064
White Callback Mean:  0.097
Black Callback std:  0.246
White Callback std:  0.295
Diff of Means:  -0.032


In [44]:
#determine the margin of error.
twosamp_std = (((black_std)**2 + (white_std)**2)/2435)**.5
marg_err = (1.96) * (twosamp_std)  
print('the margin of error is:  {0:.3f}'.format(marg_err))


the margin of error is:  0.015


In [45]:
#determine confidence interval to get the range for 95% confidence
conf_int = (diff_of_means + marg_err), (diff_of_means - marg_err)
print(conf_int)

(-0.016774437292225542, -0.047291272417895616)


In [46]:
#utilize z-score formula to determine z score
zscre = (diff_of_means - nullhyp_diff_of_means)/ (twosamp_std)
print(zscre)


-4.114738324427746


In [47]:
#import stats package to determine p value and to recheck z score above.
from statsmodels.stats.weightstats import ztest
z, pval = ztest(b['call'],w['call'], alternative='two-sided')
print('z: {} , pval: {}'.format(z, pval))

z: -4.114705356750735 , pval: 3.8767429116085706e-05


## The p value above is close to zero therefore we reject null hypothesis that there is no statistical difference between mean of black resume callbacks and white resume callbacks.

## 4.) What is a story describing the statistical significance in the context or the original problem?
## In accordance to 95% confidence interval testing, we can be 95% confident that there is a statistical difference between black and white callback.  We can also reject the null hypothesis based on p value being close to zero.  Therefore, as stated above there is a statistical difference between the mean of black resume callbacks and white resume callbacks.

## 5.)Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

### Our data analysis shows there is indeed a statistical difference in the means of black and white callbacks.  According to our data, 157 out of 2435 resume callbacks had black sounding names compared to 235 out of 2345 resume callbacks that did not.  This means it is an important factor in callback success however, we can not draw any conclusions from this since we have not looked at the other columns of the data table that determine resume callbacks. We have not analyzed if there are other factors that affect our statistical conclusions.  